In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from multi_imbalance.resampling.spider import SPIDER3
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from multi_imbalance.datasets import load_datasets
from multi_imbalance.resampling.MDO import MDO
from multi_imbalance.resampling.SOUP import SOUP
from collections import Counter
from imblearn.metrics import geometric_mean_score


In [2]:
def read_train_and_test_data(overlap, imbalance_ratio, i):
    with open(f"data/3class-{imbalance_ratio}-overlap-{overlap}-learn-{i}.arff") as f:
        content = f.readlines()
    content = [x.strip().split(",") for x in content][5:]
    data = np.array(content)
    X_train, y_train = data[:, :-1].astype(float), data[:, -1].astype(object)

    with open(f"data/3class-{imbalance_ratio}-overlap-{overlap}-test-{i}.arff") as f:
        content = f.readlines()
    content = [x.strip().split(",") for x in content][5:]
    data = np.array(content)
    X_test, y_test = data[:, :-1].astype(float), data[:, -1].astype(object)

    return X_train, y_train, X_test, y_test


def train_and_test():
    neigh = KNeighborsClassifier(n_neighbors=1)
    neigh.fit(X_train, y_train)
    y_pred = neigh.predict(X_test)
    labels = ['MIN', 'INT', 'MAJ']
    return [confusion_matrix(y_test, y_pred, labels=labels)[i, i] / confusion_matrix(y_test, y_pred, labels=labels)[i,:].sum() for i,label in enumerate(labels)]


In [ ]:
# cost = np.reshape(np.array([0, 2, 3, 3, 0, 2, 7, 5, 0]), (3, 3))
# cost = np.reshape(np.array([0, 3, 7, 2, 0, 5, 3, 2, 0]), (3, 3))
# cost = np.reshape(np.array([0, 1, 1, 3, 0, 1, 7, 5, 0]), (3, 3)) # odkopana
# cost = np.reshape(np.array([0, 1, 1, 2, 0, 1, 6, 3, 0]), (3, 3)) # try 1
# cost = np.reshape(np.array([0, 2, 6, 1, 0, 3, 1, 1, 0]), (3, 3)) # try 1
# cost = np.reshape(np.array([0, 1, 1, 1, 0, 1, 1, 1, 0]), (3, 3))


In [20]:
cost_matrix = np.array([[0,1,1],[3,0,1],[7,5,0]])

result = []


for imbalance_ratio in ["70-30-0-0", "40-50-10-0", "30-40-15-15"]:
    print(f"Imbalance ratio: {imbalance_ratio}")
    for overlap in range(0, 3):
        print(f"Overlap: {overlap}")
        for setting in ['base', 'default', 'article']:
            min_tpr = []
            int_tpr = []
            maj_tpr = []
            for i in range(1,11):
                X_train, y_train, X_test, y_test = read_train_and_test_data(overlap, imbalance_ratio, i)
                if setting == 'base':
                    pass
                elif setting == 'default':
                    cost = np.ones((3, 3))
                    for i in range(3):
                        cost[i][i] = 0

                    clf = SPIDER3(k=5, majority_classes=['MAJ'],
                                  intermediate_classes=['INT'], minority_classes=['MIN'], cost=cost)
                    X_train, y_train = clf.fit_transform(X_train.astype(np.float64), y_train)
                elif setting == 'article':
                    clf = SPIDER3(k=5, majority_classes=['MAJ'],
                                  intermediate_classes=['INT'], minority_classes=['MIN'], cost=cost_matrix)
                    X_train, y_train = clf.fit_transform(X_train.astype(np.float64), y_train)
                min_t, int_t, maj_t = train_and_test()
                min_tpr.append(min_t)
                int_tpr.append(int_t)
                maj_tpr.append(maj_t)
            print(setting, [np.array(min_tpr).mean(), np.array(int_tpr).mean(), np.array(maj_tpr).mean()])
            result.append([np.array(min_tpr).mean(), np.array(int_tpr).mean(), np.array(maj_tpr).mean()])
np.savetxt("article_results.csv", np.asarray(result), delimiter=",")

Imbalance ratio: 70-30-0-0
Overlap: 0
base [0.772, 0.8379999999999999, 0.9133333333333333]
default [0.922, 0.9640000000000001, 0.882]
article [0.9719999999999999, 0.992, 0.8556666666666667]
Overlap: 1
base [0.638, 0.792, 0.9286666666666665]
default [0.758, 0.906, 0.9013333333333335]
article [0.788, 0.922, 0.8803333333333334]
Overlap: 2
base [0.434, 0.6809999999999999, 0.9329999999999998]
default [0.532, 0.766, 0.9129999999999999]
article [0.5520000000000002, 0.779, 0.898]
Imbalance ratio: 40-50-10-0
Overlap: 0
base [0.6739999999999999, 0.7689999999999999, 0.8633333333333333]
default [0.89, 0.961, 0.76]
article [0.9319999999999998, 0.969, 0.748]
Overlap: 1
base [0.568, 0.7299999999999999, 0.8780000000000001]
default [0.74, 0.898, 0.7903333333333333]
article [0.766, 0.907, 0.7776666666666667]
Overlap: 2
base [0.45600000000000007, 0.702, 0.8953333333333333]
default [0.656, 0.8379999999999999, 0.8103333333333333]
article [0.6639999999999999, 0.843, 0.8020000000000002]
Imbalance ratio: 30-4

In [21]:
pd.read_csv("article_results.csv", header=None)

,0,1,2
0,0.772,0.838,0.913333
1,0.922,0.964,0.882000
2,0.972,0.992,0.855667
3,0.638,0.792,0.928667
4,0.758,0.906,0.901333
5,0.788,0.922,0.880333
6,0.434,0.681,0.933000
7,0.532,0.766,0.913000
8,0.552,0.779,0.898000
9,0.674,0.769,0.863333


In [5]:
result = []


for imbalance_ratio in ["70-30-0-0", "40-50-10-0", "30-40-15-15"]:
    print(f"Imbalance ratio: {imbalance_ratio}")
    for overlap in range(0, 3):
        print(f"Overlap: {overlap}")
        for setting in ['base', 'default', 'custom']:
            min_tpr = []
            int_tpr = []
            maj_tpr = []
            for i in range(1,11):
                X_train, y_train, X_test, y_test = read_train_and_test_data(overlap, imbalance_ratio, i)
                if setting == 'base':
                    pass
                elif setting == 'default':
                    cost = np.ones((3, 3))
                    for i in range(3):
                        cost[i][i] = 0

                    clf = SPIDER3(k=5, majority_classes=['MAJ'],
                                  intermediate_classes=['INT'], minority_classes=['MIN'], cost=cost)
                    X_train, y_train = clf.fit_transform(X_train.astype(np.float64), y_train)
                elif setting == 'custom':
                    clf = SPIDER3(k=5, majority_classes=['MAJ'],
                                  intermediate_classes=['INT'], minority_classes=['MIN'])
                    X_train, y_train = clf.fit_transform(X_train.astype(np.float64), y_train)
                min_t, int_t, maj_t = train_and_test()
                min_tpr.append(min_t)
                int_tpr.append(int_t)
                maj_tpr.append(maj_t)
            print(setting, [np.array(min_tpr).mean(), np.array(int_tpr).mean(), np.array(maj_tpr).mean()])
            result.append([np.array(min_tpr).mean(), np.array(int_tpr).mean(), np.array(maj_tpr).mean()])
np.savetxt("spider-results.csv", np.asarray(result), delimiter=",")

Imbalance ratio: 70-30-0-0
Overlap: 0
base [0.772, 0.8379999999999999, 0.9133333333333333]
default [0.922, 0.9640000000000001, 0.882]
custom [0.97, 0.9710000000000001, 0.8666666666666666]
Overlap: 1
base [0.638, 0.792, 0.9286666666666665]
default [0.758, 0.906, 0.9013333333333335]
custom [0.788, 0.9030000000000001, 0.8936666666666667]
Overlap: 2
base [0.434, 0.6809999999999999, 0.9329999999999998]
default [0.532, 0.766, 0.9129999999999999]
custom [0.55, 0.7699999999999999, 0.9056666666666666]
Imbalance ratio: 40-50-10-0
Overlap: 0
base [0.6739999999999999, 0.7689999999999999, 0.8633333333333333]
default [0.89, 0.961, 0.76]
custom [0.9319999999999998, 0.9450000000000001, 0.7576666666666666]
Overlap: 1
base [0.568, 0.7299999999999999, 0.8780000000000001]
default [0.74, 0.898, 0.7903333333333333]
custom [0.768, 0.897, 0.7863333333333333]
Overlap: 2
base [0.45600000000000007, 0.702, 0.8953333333333333]
default [0.656, 0.8379999999999999, 0.8103333333333333]
custom [0.6639999999999999, 0.83

In [8]:
df = pd.read_csv("spider-results.csv", header=None)

### Experiments on synthetic dataset

In [9]:
df

,0,1,2
0,0.772,0.838,0.913333
1,0.922,0.964,0.882000
2,0.970,0.971,0.866667
3,0.638,0.792,0.928667
4,0.758,0.906,0.901333
5,0.788,0.903,0.893667
6,0.434,0.681,0.933000
7,0.532,0.766,0.913000
8,0.550,0.770,0.905667
9,0.674,0.769,0.863333


In [10]:
datasets = load_datasets()

In [13]:
datasets.keys()

odict_keys(['1czysty-cut', '2delikatne-cut', '3mocniej-cut', '4delikatne-bezover-cut', 'balance-scale', 'cleveland', 'cleveland_v2', 'cmc', 'dermatology', 'glass', 'hayes-roth', 'new_ecoli', 'new_led7digit', 'new_vehicle', 'new_winequality-red', 'new_yeast', 'thyroid-newthyroid'])

In [18]:
for k in datasets.keys():
    print(k, Counter(datasets[k].target))

1czysty-cut Counter({0: 840, 2: 240, 1: 120})
2delikatne-cut Counter({0: 840, 2: 240, 1: 120})
3mocniej-cut Counter({0: 840, 2: 240, 1: 120})
4delikatne-bezover-cut Counter({0: 840, 2: 240, 1: 120})
balance-scale Counter({2: 288, 1: 288, 0: 49})
cleveland Counter({0: 164, 1: 55, 2: 36, 3: 35, 4: 13})
cleveland_v2 Counter({0: 219, 1: 36, 2: 35, 3: 13})
cmc Counter({0: 629, 2: 511, 1: 333})
dermatology Counter({0: 112, 2: 72, 1: 61, 4: 52, 3: 49, 5: 20})
glass Counter({1: 76, 0: 70, 3: 29, 5: 17, 2: 13, 4: 9})
hayes-roth Counter({0: 65, 1: 64, 2: 31})
new_ecoli Counter({0: 145, 1: 77, 4: 52, 2: 37, 3: 25})
new_led7digit Counter({3: 108, 5: 99, 0: 98, 2: 94, 4: 52, 1: 49})
new_vehicle Counter({1: 429, 0: 218, 2: 199})
new_winequality-red Counter({0: 681, 1: 638, 2: 199, 3: 81})
new_yeast Counter({0: 463, 1: 429, 8: 244, 7: 168, 6: 51, 5: 44, 4: 35, 3: 30, 2: 20})
thyroid-newthyroid Counter({0: 150, 1: 35, 2: 30})


In [ ]:
maj_int_min = {
    "1czysty-cut": {'maj': [0], 'int': [2], 'min': [1]},
    "2delikatne-cut": {'maj': [0], 'int': [2], 'min': [1]},
    "3mocniej-cut": {'maj': [0], 'int': [2], 'min': [1]},
    "4delikatne-bezover-cut": {'maj': [0], 'int': [2], 'min': [1]},
    "balance-scale": {'maj': [2, 1], 'int': [], 'min': [0]},
    "cleveland": {'maj': [0], 'int': [], 'min': [1, 2, 3, 4]},
    "cleveland_v2": {'maj': [0], 'int': [], 'min': [1, 2, 3]},
    "cmc": {'maj': [0, 2], 'int': [], 'min': [1]},
    "dermatology": {'maj': [0, 2, 1, 4, 3], 'int': [], 'min': [5]},
    "glass": {'maj': [1, 0, 3], 'int': [], 'min': [5, 2, 4]},
    "hayes-roth": {'maj': [0, 1], 'int': [], 'min': [2]},
    "new_ecoli": {'maj': [0, 1], 'int': [], 'min': [4, 2, 3]},
    "new_led7digit": {'maj': [3, 5, 0, 2], 'int': [], 'min': [4, 1]},
    "new_vehicle": {'maj': [1], 'int': [], 'min': [0, 2]},
    "new_winequality-red": {'maj': [0, 1], 'int': [], 'min': [2, 3]},
    "new_yeast": {'maj': [0, 1, 8, 7], 'int': [], 'min': [6, 5, 4, 3, 2]},
    "thyroid-newthyroid": {'maj': [0], 'int': [], 'min': [1, 2]}
}